In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, ConstantKernel, RBF
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [84]:
dir = 'C:/Users/fitzpatrick/Desktop/Data/Input/'

## Read in PCP data from CFSR
data_1 = pd.read_csv(dir+'CFSR_APCP_Basin_Avgs.csv',sep=',')
#data_1 = data_1.iloc[:-n_for]
month = data_1['month']
su_pcp = data_1['WaterSuperior']
er_pcp = data_1['WaterErie']
on_pcp = data_1['WaterOntario']
mh_pcp = data_1['WaterMichigan']+data_1['WaterHuron']

## Read in EVAP data from CFSR
data_2 = pd.read_csv(dir+'CFSR_EVAP_Basin_Avgs.csv',sep=',')
#data_2 = data_2.iloc[:-n_for]
su_evap = data_2['WaterSuperior']
er_evap = data_2['WaterErie']
on_evap = data_2['WaterOntario']
mh_evap = data_2['WaterMichigan']+data_2['WaterHuron']

## Read in TMP data from CFSR
data_3 = pd.read_csv(dir+'CFSR_TMP_Basin_Avgs.csv',sep=',')
#data_3 = data_3.iloc[:-n_for]
su_tmp = data_3['WaterSuperior']
er_tmp = data_3['WaterErie']
on_tmp = data_3['WaterOntario']
mh_tmp = data_3['WaterMichigan']+data_3['WaterHuron']

In [85]:
# Read in GLCC RNBS data
data_4 = pd.read_csv(dir + 'rnbs_glcc.csv', sep=',')

# Ensure 'Date' column is treated as string
date_strs = data_4['Date'].astype(str)

date_rnbs = []

for date_str in date_strs:
    # Insert leading zero before month
    if len(date_str) == 6:
        date_str = date_str[:4] + '0' + date_str[4:]

    # Insert leading zero before day
    if len(date_str) == 7:
        date_str = date_str[:6] + '0' + date_str[6:]

    # Convert to datetime object
    datetime_obj = datetime.strptime(date_str, '%Y%m%d')

    # Format the datetime object as a string in the desired format
    date_tmp = datetime_obj.strftime('%Y-%m-%d')

    # Append formatted date to list
    date_rnbs.append(date_tmp)

# Replace the original 'Date' column with formatted dates
data_4['Date'] = date_rnbs

# Convert 'Date' column to datetime format
data_4['Date'] = pd.to_datetime(data_4['Date'])

# Add a new column 'Month' extracting month from 'Date'
data_4['Month'] = data_4['Date'].dt.month

# Select only the data that matches with the CFSR dates
start_date = '1979-01-01'
end_date = '2010-12-01'
selected_data = data_4[(data_4['Date'] >= start_date) & (data_4['Date'] <= end_date)]

su_rnbs = selected_data['sup']
er_rnbs = selected_data['eri']
on_rnbs = selected_data['ont']
mh_rnbs = selected_data['mic_hur']

In [87]:
# Prepare the data 
# Features
X = pd.DataFrame({
    'su_pcp': su_pcp,
    'er_pcp': er_pcp,
    'on_pcp': on_pcp,
    'mh_pcp': mh_pcp,
    'su_evap': su_evap,
    'er_evap': er_evap,
    'on_evap': on_evap,
    'mh_evap': mh_evap,
    'su_tmp': su_tmp,
    'er_tmp': er_tmp,
    'on_tmp': on_tmp,
    'mh_tmp': mh_tmp
})

# Targets
y = pd.DataFrame({
    'su_rnbs': su_rnbs,
    'er_rnbs': er_rnbs,
    'on_rnbs': on_rnbs,
    'mh_rnbs': mh_rnbs
})
#y = targets_df

## IF we want to include the month as a categorical feature
# Merge on 'Month'
#X = pd.concat([X, pd.get_dummies(month, prefix='Month')], axis=1)

# Drop any rows with NaN values (if any)
#X.dropna(inplace=True)

In [88]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [89]:
# Gaussian Process Regression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, ConstantKernel, RBF

kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)

# Fit the model
gpr.fit(X_train, y_train)

# Predictions
y_pred, sigma = gpr.predict(X_test, return_std=True)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
print(f"The r squared value for the model is {r_squared}")
print(f"Mean Squared Error: {mse}")

The r squared value for the model is 0.5532045537319937
Mean Squared Error: 3553.5639059802224


c:\Users\fitzpatrick\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [90]:
## Random Forest Regressor Model
import joblib

# Initialize RandomForestRegressor
model = RandomForestRegressor(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Save the trained model
joblib.dump(model, 'RF_trained_model.pkl')

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
print(f"The r squared value for the model is {r_squared}")
print(f"Mean Squared Error: {mse}")

The r squared value for the model is 0.7512092173862659
Mean Squared Error: 2011.782449766933


In [91]:
# Get feature importances
importances = model.feature_importances_
feature_names = X.columns

# Create a DataFrame to display feature importances
feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# Sort features by importance
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# Display the feature importances
print(feature_importances_df)

    Feature  Importance
5   er_evap    0.179492
6   on_evap    0.170444
7   mh_evap    0.149720
1    er_pcp    0.127197
2    on_pcp    0.075403
10   on_tmp    0.056420
4   su_evap    0.054139
11   mh_tmp    0.047246
3    mh_pcp    0.041353
9    er_tmp    0.040175
0    su_pcp    0.030408
8    su_tmp    0.028003


In [92]:
## Neural Network

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4)  # Number of targets
])

# Compile the model
model.compile(optimizer='adam', loss='mse')  # Using mean squared error (mse) as the loss function

# Fit the model to the training data
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Save the trained model
model.save('NN_trained_model.keras')

# Predict on the test set
y_pred = model.predict(X_test_scaled)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
print(f"The r squared value for the model is {r_squared}")
print(f"Mean Squared Error: {mse}")

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 22022.6992 - val_loss: 14440.4834
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 20078.2402 - val_loss: 14360.1514
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 20475.2695 - val_loss: 14252.6436
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 21235.3984 - val_loss: 14106.1738
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 18602.9375 - val_loss: 13909.2842
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 20020.3105 - val_loss: 13640.3975
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 18840.4902 - val_loss: 13287.8262
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 20283.5625 - val_loss: 12829.9561
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 17526.5586 - val_loss: 12252.1631
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 16881.6836 - val_loss: 11536.4883
Epoch 11/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 16395.7676 - val_loss: 10695.9766
Epoc